In [ ]:
import utils.utils as utils
import app
import ipywidgets as widgets
from IPython.display import display

source_language = "it"
target_language = "en"

In [ ]:
# Create a button for file selection
file_paths = []
button = widgets.Button(description="Browse Files")
output = widgets.Output()


def on_button_clicked(b):
    with output:
        global file_paths
        file_paths = utils.select_files()


button.on_click(on_button_clicked)
display(button, output)

In [ ]:
service = "openai"
for file_path in file_paths:
    srt_response1 = app.transcribe_translate(
        file_path=file_path,
        service_name=service,
        source_language=source_language,
        target_language=target_language,
    )
    print(srt_response1)
    download_path = utils.replace_extension(
        file_path=file_path, end_modifiers=f"_{service}", new_extension=".srt"
    )
    utils.save_srt_data(srt_data=srt_response1, file_path=download_path)

In [ ]:
service = "google"
for file_path in file_paths:
    srt_response2 = app.transcribe_translate(
        file_path=file_path,
        service_name=service,
        source_language=source_language,
        target_language=target_language,
    )
    print(srt_response2)
    download_path = utils.replace_extension(
        file_path=file_path, end_modifiers=f"_{service}", new_extension=".srt"
    )
    utils.save_srt_data(srt_data=srt_response2, file_path=download_path)

In [ ]:
# multiple concurrent transcriptions
services = ["openai", "google"]
for file_path in file_paths:
    all_responses = app.multi_transcribe(
        file_path=file_path,
        service_names=services,
        source_language=source_language,
        target_language=target_language,
        audio_output_extension=".mp3",
    )
    for service, srt_response in all_responses.items():
        print(f"{service} {srt_response}")
        download_path = utils.replace_extension(
            file_path=file_path, end_modifiers=f"_{service}", new_extension=".srt"
        )
        utils.save_srt_data(srt_data=srt_response, file_path=download_path)